In [ ]:
import pandas as pd
import numpy as np
import random
from rdkit import Chem
from snn_model import SNNet, device, batch_size, train_snn, test_snn, get_loss_fn
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
#from snntorch import spikegen, surrogate
import matplotlib.pyplot as plt
from utils import load_dataset_df, fp_generator
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, f1_score, precision_score

import deepchem as dc


No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\knsve\Desktop\MEI\Tese\torch\pt_venv2\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (c:\Users\knsve\Desktop\MEI\Tese\torch\pt_venv2\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
#!pip install pandas numpy rdkit torch snntorch matplotlib scikit-learn deepchem pubchempy

In [ ]:
#https://github.com/chainer/chainer-chemistry/blob/master/chainer_chemistry/dataset/splitters/scaffold_splitter.py

#splitter = ScaffoldSplitter()
#tasks, datasets, transformers = dc.molnet.load_bbbp(splitter=splitter, featurizer="ECFP", reload=True)
#train, val, test = datasets

#### Load DataFrame

In [30]:
files = ['tox21.csv','sider.csv', 'BBBP.csv']
dt_file = files[2]

df, targets = load_dataset_df(filename=dt_file)
print(targets)

target_name = targets[0]
df = df[[target_name, 'smiles']].dropna()


['p_np']


#### SMILE to Fingerprint

In [ ]:
fp_types = [['morgan', 1024], ['maccs', 167], ['RDKit', 1024], ['pubchem', 881]]
fp_type, num_bits = fp_types[0]
num_bits = 237
print(fp_type, '-', num_bits)
num_rows = len(df)
fp_array = np.zeros((num_rows, num_bits))
target_array = np.zeros((num_rows, 1))
i = 0

img = None
# Smile to Fingerprint of size {num_bits}
fp_gen = fp_generator(fp_type, fp_size=num_bits, radius=2)

for idx, row in df.iterrows():
    mol = Chem.MolFromSmiles(row['smiles'])
    #TODO: sanitize molecules to remove the warnings (?)
    
    if mol is None:
        continue
    
    if fp_type == "pubchem":
        fingerprint = fp_gen(row['smiles'])
        if fingerprint is None:
            continue
    else:
        fingerprint = fp_gen(mol)

    fp_array[i] = np.array(fingerprint)
    target_array[i] = row[target_name]
    i += 1
target_array = target_array.ravel()
print(np.sum(fp_array[i + 1]))
fp_array = fp_array[0:i]
target_array = target_array[0:i]

In [ ]:
print(fp_array.shape)

In [ ]:
# Create Torch Dataset
dtype = torch.float32
fp_tensor = torch.tensor(fp_array, dtype=dtype)
target_tensor = torch.tensor(target_array, dtype=dtype).long()

dataset = TensorDataset(fp_tensor, target_tensor)

#### Loss Function

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

loss_types = ['cross_entropy', 'rate_loss', 'temporal_loss']
loss_type = loss_types[0]
print(loss_type)

use_weights = True


#### Train Loop

In [ ]:
num_epochs = 1000 #aumentar e diminuir lr
lr=1e-4 #default was e5-5
iterations = 1
time_steps = 10
num_hidden = 256 # tentar 128

In [ ]:
net = SNNet(input_size=num_bits,num_hidden=num_hidden, num_steps=10, spike_grad=None, use_l2=False).to(device)

num_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(f"Number of trainable parameters SNN: {num_params}")

net = SNNet(input_size=num_bits,num_hidden=num_hidden,num_steps=10, spike_grad=None, use_l2=True).to(device)

num_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(f"Number of trainable parameters DSNN: {num_params}")


In [ ]:
results = [[], [], [], [], [], []]
for iter in range(iterations):
    print(f"Iteration:{iter + 1}/{iterations}")
    random.seed(iter)
    #spike_grad=surrogate.fast_sigmoid()
    spike_grad=None
    net = SNNet(input_size=num_bits,num_hidden=num_hidden,num_steps=time_steps, spike_grad=spike_grad, use_l2=False).to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=0) #experimentar lr diferente
    #experimentar weight decay = 1e-4 e subir (pouco, 1e-3)

    generator = torch.Generator().manual_seed(iter + 1)
    #train, test = random_split(dataset, [0.7, 0.3], generator=generator) #experimentar 0.9, 0.1
    #train, val, test = random_split(dataset, [0.8, 0.1, 0.1], generator=generator) #experimentar 0.9, 0.1
    splitter = ScaffoldSplitter()
    train, val, test = dc.molnet.load_bbbp(splitter=splitter)
    _, train_label = train[:]
    _, val_label = val[:]
    _, test_label = test[:]

    #print("positive labels in the test set:", int(test_label.sum()))
    #print("negative labels in the test set:", int(len(test_label) - test_label.sum()))
    # Load the Data
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)

    #Loss Function
    if use_weights:
        class_weights = compute_class_weight(class_weight='balanced', classes=np.array([0, 1]), y=np.array(train_label))
        #class_weights[0], class_weights[1] = class_weights[1], class_weights[0]
        class_weights[0] += 5
        class_weights = torch.tensor(class_weights, dtype=torch.float)
        weighted = 'class_weights'
    else: 
        class_weights=None
        weighted = ''

    loss_fn = get_loss_fn(loss_type=loss_type, class_weights=class_weights)

    net, loss_hist = train_snn(net, optimizer, device, num_epochs, train_loader, val_loader, loss_type, loss_fn, dtype)

    #fig = plt.figure(facecolor="w", figsize=(10, 5))
    #plt.plot(loss_hist)
    #plt.title("Loss Curve")
    #plt.xlabel("Iteration")
    #plt.ylabel("Loss")
    #plt.show()

    all_preds, all_targets = test_snn(net, device, test_loader)

    accuracy = accuracy_score(all_targets, all_preds)
    auc_roc = roc_auc_score(all_targets, all_preds)
    tn, fp, fn, tp = confusion_matrix(all_targets, all_preds).ravel()
    sensitivity = tp/(tp + fn)
    specificity = tn/(tn + fp)
    f1 = f1_score(all_targets, all_preds)
    precision = precision_score(all_targets, all_preds)

    results[0].append(accuracy)
    results[1].append(auc_roc)
    results[2].append(sensitivity)
    results[3].append(specificity)
    results[4].append(f1)
    results[5].append(precision)


#### Smoothed Loss

In [ ]:
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter1d
from statsmodels.nonparametric.smoothers_lowess import lowess

print(loss_hist[len(loss_hist) - 5:len(loss_hist)])

fig = plt.figure(facecolor="w", figsize=(10, 5))
#plt.plot(np.convolve(loss_hist, np.ones(30)/30, mode='valid'))
#plt.plot(savgol_filter(loss_hist, window_length=100, polyorder=3))
#plt.plot(lowess(loss_hist, np.arange(len(loss_hist)), frac=0.1)[:, 1])
plt.plot(gaussian_filter1d(loss_hist, sigma=6))
#plt.plot(loss_hist)
plt.axhline(y=1, color='r', linestyle='--', label='y = 1')
plt.title("Loss Curve")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# Validation Set
# ROC auc plot every epochs


#### Save Metrics

In [ ]:
metrics_np = np.zeros(12)

for i, metric in enumerate(results):
    metrics_np[i*2] = np.round(np.mean(metric), 3)
    metrics_np[i*2+1] = np.round(np.std(metric), 3)

# Print Results
print(f"Accuracy:  {metrics_np[0]:.3f} ± {metrics_np[1]:.3f}")
print(f"AUC ROC: {metrics_np[2]:.3f} ± {metrics_np[3]:.3f}")
print(f"Sensitivity: {metrics_np[4]:.3f} ± {metrics_np[5]:.3f}")
print(f"Specificity: {metrics_np[6]:.3f} ± {metrics_np[7]:.3f}")

metric_names = ['Acc', 'AUC', 'Sn', 'Sp', 'F1', 'Precision']
metrics_np = metrics_np.reshape(1, -1)
columns = []
for name in metric_names:
    columns.extend([f'Mean {name}', f'Std {name}'])


df_metrics = pd.DataFrame(metrics_np, columns=columns)
dirname = dt_file.strip('.csv')

#adicionar lr
filename = f"results\\{dirname}\\{target_name}_{fp_type}_{num_bits}_{num_hidden}_{time_steps}_lr{lr}_e{num_epochs}_{loss_type}_{weighted}.csv"
#df_metrics.to_csv(filename, index=False)

print(filename)